In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import pickle

In [2]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [3]:
vote_file['Party abbreviation'].value_counts()

Con      333
Lab      238
SNP       56
LD         9
DUP        8
SF         4
PC         3
SDLP       3
UUP        2
Ind        1
UKIP       1
Spk        1
Green      1
Name: Party abbreviation, dtype: int64

In [4]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
vocab = vocab[:2000]

In [5]:
politicians_party = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']


In [7]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-6-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df = df.loc[df['Topic'] == 'European Union']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2']
df.columns = cols
df.head()

,date,speaker,name,Party,Topic,transcript,party2
21076,2015-06-09,The Secretary of State for Foreign and Commonw...,Philip Hammond,Conservative,European Union,"I beg to move, That the Bill be now read a Sec...",0.0
21078,2015-06-09,Mr Hammond,Philip Hammond,Conservative,European Union,"I am going to make a little progress, bearing ...",0.0
21079,2015-06-09,Mr Kenneth Clarke (Rushcliffe) (Con),Kenneth Clarke,Conservative,European Union,There must have been some strange juxtapositio...,0.0
21080,2015-06-09,Mr Hammond,Philip Hammond,Conservative,European Union,"Call me negligent, but as an 18-year-old voter...",0.0
21082,2015-06-09,Mr Hammond,Philip Hammond,Conservative,European Union,If the hon. Gentleman can bear to stop wagging...,0.0


In [8]:
df.to_csv('Parliament_speeches_June_2015-June_2016_EU.csv', index=False)

In [243]:
topics = df['Topic'].unique()
for i in range(len(topics)):
    if '|' in topics[i]:
        topics[i] = topics[i].split('|')[0].strip()
    else:
        topics[i] = topics[i].strip()
        
topics = list(set(topics))
topics.remove('admin')
topics.remove('Others')
topics

['Culture, media and sport',
 'Energy and environment',
 'European Union',
 'Health services and medicine',
 'Defence',
 'Science and technology',
 'Transport',
 'Asylum, immigration and nationality',
 'Economy and finance',
 'Communities and families',
 'Crime, civil law, justice and rights',
 'Agriculture, animals, food and rural affairs',
 'Business, industry and consumers',
 'Employment and training',
 'Housing and planning',
 'International affairs',
 'Social security and pensions',
 'Parliament, government and politics',
 'Education',
 'Social services']

In [244]:
topics_map = {
    'Culture, media and sport' : 'culture_media',
    'Energy and environment' : 'environ',
    'European Union' : 'eu',
    'Health services and medicine' : 'health',
    'Defence' : 'defence',
    'Science and technology' : 'science',
    'Transport' : 'transport',
    'Asylum, immigration and nationality' : 'asylum',
    'Parliament, government and politics' : 'parliament'
}
topics = list(topics_map.keys())

In [226]:
for topic in topics:
    df_topic = df.loc[df['Topic'] == topic]
    
    leave_conservative = df_topic.loc[(df_topic['Party'] == 'Conservative') & (df_topic['party2'] == 1)]
    remain_labour = df_topic.loc[(df_topic['Party'] == 'Labour') & (df_topic['party2'] == 0)]
    remain_conservative = df_topic.loc[(df_topic['Party'] == 'Conservative') & (df_topic['party2'] == 0)]

    # leave_conservative = leave_conservative.sample(100)
    # remain_labour = remain_labour.sample(100)
    # remain_conservative = remain_conservative.sample(100)

    lines = []
    cols = ["text", "text_classification_label"]

    for index, row in leave_conservative.iterrows():
        sentences = nltk.sent_tokenize(row['transcript'])

        for sent in sentences:
            if len(sent.split()) < 11:
                continue
            lines.append([sent, "leave_conservative"])

    res_df1 = pd.DataFrame(lines, columns=cols)
    res_df1 = res_df1.sample(min(1000, len(res_df1)))

    lines = []
    for index, row in remain_labour.iterrows():
        sentences = nltk.sent_tokenize(row['transcript'])

        for sent in sentences:
            if len(sent.split()) < 11:
                continue
            lines.append([sent, "remain_labour"])

    res_df2 = pd.DataFrame(lines, columns=cols)
    res_df2 = res_df2.sample(min(1000, len(res_df2)))
    res_df = pd.concat([res_df1, res_df2])

    res_df = res_df.sample(frac=1)
    res_df.to_csv('train_{}.tsv'.format(topics_map[topic]), sep='\t', index=False)

    lines = []
    cols = ["text"]

    for index, row in remain_conservative.iterrows():
        sentences = nltk.sent_tokenize(row['transcript'])
        for sent in sentences:
            if len(sent.split()) < 11:
                continue
            lines.append(sent)

    res_df = pd.DataFrame(lines, columns=cols)
    res_df.to_csv('test_{}.tsv'.format(topics_map[topic]), sep='\t', index=False)

In [246]:
leave_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 1)]
remain_labour = df.loc[(df['Party'] == 'Labour') & (df['party2'] == 0)]
remain_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 0)]

lines = []
for index, row in leave_conservative.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    lines.extend(sentences)
    lines.append('\n\n')
    
with open('leave_conservative.txt', 'w') as f:
    f.writelines(lines)
    
lines = []
for index, row in remain_labour.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    lines.extend(sentences)
    lines.append('\n\n')
    
with open('remain_labour.txt', 'w') as f:
    f.writelines(lines)
    
lines = []
for index, row in remain_conservative.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    lines.extend(sentences)
    lines.append('\n\n')
    
with open('remain_conservative.txt', 'w') as f:
    f.writelines(lines)
    

In [247]:
leave_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 1)]
remain_labour = df.loc[(df['Party'] == 'Labour') & (df['party2'] == 0)]
remain_conservative = df.loc[(df['Party'] == 'Conservative') & (df['party2'] == 0)]

# leave_conservative = leave_conservative.sample(100)
# remain_labour = remain_labour.sample(100)
# remain_conservative = remain_conservative.sample(100)

lines = []
cols = ["text", "text_classification_label"]

for index, row in leave_conservative.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    
    for sent in sentences:
        if len(sent.split()) < 11:
            continue
        lines.append([sent, "leave_conservative"])

res_df1 = pd.DataFrame(lines, columns=cols)

lines = []
for index, row in remain_labour.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    
    for sent in sentences:
        if len(sent.split()) < 11:
            continue
        lines.append([sent, "remain_labour"])

res_df2 = pd.DataFrame(lines, columns=cols)
res_df = pd.concat([res_df1, res_df2])

res_df = res_df.sample(frac=1)
res_df.to_csv('train.tsv', sep='\t', index=False)
    
lines = []
cols = ["text"]

for index, row in remain_conservative.iterrows():
    sentences = nltk.sent_tokenize(row['transcript'])
    for sent in sentences:
        if len(sent.split()) < 11:
            continue
        lines.append(sent)
    
res_df = pd.DataFrame(lines, columns=cols)
res_df.to_csv('test.tsv', sep='\t', index=False)

In [36]:
import logging
from pathlib import Path

from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import BertStyleLMProcessor
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import BertLMHead, NextSentenceHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.modeling.optimization import initialize_optimizer

from farm.utils import set_all_seeds, MLFlowLogger, initialize_device_settings

# Train Bert classification

In [ ]:
def lm_finetuning():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    next_sent_pred_style = "bert-style"
    next_sent_pred=True
    set_all_seeds(seed=42)
    ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
    ml_logger.init_experiment(
        experiment_name="LM_refactoring", run_name=f"new, nsp: {next_sent_pred}, {next_sent_pred_style}"
    )
    ##########################
    ########## Settings
    ##########################
    device, n_gpu = initialize_device_settings(use_cuda=True)
    n_epochs = 1
    batch_size = 32
    evaluate_every = 1000
    lang_model = "bert-base-cased"
    do_lower_case = False

    # 1.Create a tokenizer
    tokenizer = Tokenizer.load(
        pretrained_model_name_or_path=lang_model, do_lower_case=do_lower_case
    )
    
    tokenizer.add_tokens(vocab)
    # 2. Create a DataProcessor that handles all the conversion from raw text into a pytorch Dataset
    processor = BertStyleLMProcessor(
        data_dir=Path("."),
        train_filename="remain_conservative.txt",
        test_filename="leave_conservative.txt",
        dev_filename=None,
        tokenizer=tokenizer,
        max_seq_len=256,
        next_sent_pred=False
    )

    # 3. Create a DataSilo that loads several datasets (train/dev/test), provides DataLoaders for them and calculates a few descriptive statistics of our datasets
    data_silo = DataSilo(processor=processor, batch_size=batch_size, max_multiprocessing_chunksize=20)

    # 4. Create an AdaptiveModel
    # a) which consists of a pretrained language model as a basis
#     language_model = LanguageModel.load(lang_model)
    language_model = LanguageModel.from_scratch("bert", tokenizer.vocab_size, n_added_tokens=len(tokenizer.get_added_vocab()))
    # b) and *two* prediction heads on top that are suited for our task => Language Model finetuning
    lm_prediction_head = BertLMHead.load(lang_model)

    model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[lm_prediction_head],
        embeds_dropout_prob=0.1,
        lm_output_types=["per_token", "per_sequence"],
        device=device,
    )

    # 5. Create an optimizer
    model, optimizer, lr_schedule = initialize_optimizer(
        model=model,
        learning_rate=2e-5,
        device=device,
        n_batches=len(data_silo.loaders["train"]),
        n_epochs=n_epochs,
        schedule_opts={'name': 'CosineWarmup', 'warmup_proportion': 0.1}
    )

    # 6. Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device
    )

    # 7. Let it grow! Watch the tracked metrics live on the public mlflow server: https://public-mlflow.deepset.ai
    trainer.train()

    # 8. Hooray! You have a model. Store it:
    save_dir = Path("saved_models/bert-english-lm-tutorial")
    model.save(save_dir)
    processor.save(save_dir)

In [ ]:
lm_finetuning()

# Test Bert Classfication

In [250]:
from pathlib import Path
import logging
import numpy as np

from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import TextClassificationProcessor
from farm.modeling.optimization import initialize_optimizer
from farm.infer import Inferencer
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import DistilBert
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.utils import set_all_seeds, initialize_device_settings


def test_doc_classification():
    set_all_seeds(seed=42)
    device, n_gpu = initialize_device_settings(use_cuda=False)
    n_epochs = 1
    batch_size = 32
    evaluate_every = 1000
    lang_model = "distilbert-base-german-cased"

    tokenizer = Tokenizer.load(
        pretrained_model_name_or_path=lang_model,
        do_lower_case=False)

    processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=128,
                                            data_dir=Path("."),
                                            train_filename=Path("train.tsv"),
                                            label_list=["leave_conservative", "remain_labour"],
                                            metric="f1_macro",
                                            test_filename=None,
                                            dev_split=0.2,
                                            label_column_name="text_classification_label")

    data_silo = DataSilo(
        processor=processor,
        batch_size=batch_size)

    language_model = DistilBert.load(lang_model)
    prediction_head = TextClassificationHead(num_labels=2)
    model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=0.1,
        lm_output_types=["per_sequence"],
        device=device)

    model, optimizer, lr_schedule = initialize_optimizer(
        model=model,
        learning_rate=2e-5,
        n_batches=len(data_silo.loaders["train"]),
        n_epochs=1,
        device=device,
        schedule_opts=None)

    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device)

    trainer.train()

    save_dir = Path("testsave/doc_class")
    model.save(save_dir)
    processor.save(save_dir)

    del model
    del processor
    del optimizer
    del data_silo
    del trainer

    basic_texts = [
        {"text": "Malte liebt Berlin."},
        {"text": "Schartau sagte dem Tagesspiegel, dass Fischer ein Idiot sei."}
    ]

    inf = Inferencer.load(save_dir, batch_size=2, num_processes=0)
    result = inf.inference_from_dicts(dicts=basic_texts)
    assert isinstance(result[0]["predictions"][0]["probability"], np.float32)
    del inf

In [251]:
test_doc_classification()

06/22/2021 14:34:44 - INFO - farm.utils -   Using device: CPU 
06/22/2021 14:34:44 - INFO - farm.utils -   Number of GPUs: 0
06/22/2021 14:34:44 - INFO - farm.utils -   Distributed Training: False
06/22/2021 14:34:44 - INFO - farm.utils -   Automatic Mixed Precision: None
06/22/2021 14:34:45 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'DistilBertTokenizer'
06/22/2021 14:34:47 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
06/22/2021 14:34:47 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
06/22/2021 14:34:47 - INFO - farm.data_handler.data_silo -   LOADING TRAIN DATA
06/22/2021 14:34:47 - INFO - farm.data_handler.data_silo -   ==================
06/22/2021 14:34:47 - INFO - farm.data_handler.data_silo -   Loading train set from: trai

06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   =================
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   Loading dev set as a slice of train set
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   Took 36844 samples out of train set to create dev set (dev split is roughly 0.2)
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   LOADING TEST DATA
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   =================
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   No test set is being loaded
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   DATASETS SUMMARY
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   ================
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   Examples in train: 141819
06/22/2021 14:35:09 - INFO - farm.data_handler.data_silo -   Exa

Train epoch 0/0 (Cur. train loss: 0.6769):   1%|          | 50/4432 [04:30<6:17:14,  5.17s/it]









Train epoch 0/0 (Cur. train loss: 0.6492):   1%|          | 50/4432 [04:30<6:17:14,  5.17s/it]









Train epoch 0/0 (Cur. train loss: 0.6492):   1%|          | 51/4432 [04:35<6:11:51,  5.09s/it]









Train epoch 0/0 (Cur. train loss: 0.6400):   1%|          | 51/4432 [04:35<6:11:51,  5.09s/it]









Train epoch 0/0 (Cur. train loss: 0.6400):   1%|          | 52/4432 [04:40<6:12:18,  5.10s/it]









Train epoch 0/0 (Cur. train loss: 0.6558):   1%|          | 52/4432 [04:40<6:12:18,  5.10s/it]









Train epoch 0/0 (Cur. train loss: 0.6558):   1%|          | 53/4432 [04:46<6:14:13,  5.13s/it]









Train epoch 0/0 (Cur. train loss: 0.6805):   1%|          | 53/4432 [04:46<6:14:13,  5.13s/it]









Train epoch 0/0 (Cur. train loss: 0.6805):   1%|          | 54/4432 [04:51<6:13:01,  5.11s/it]









Train epoch 0/0 (Cur. train loss: 0.6541):   1%|          | 54/4

Train epoch 0/0 (Cur. train loss: 0.6843):   3%|▎         | 111/4432 [09:47<6:10:10,  5.14s/it]









Train epoch 0/0 (Cur. train loss: 0.7128):   3%|▎         | 111/4432 [09:47<6:10:10,  5.14s/it]









Train epoch 0/0 (Cur. train loss: 0.7128):   3%|▎         | 112/4432 [09:52<6:12:45,  5.18s/it]









Train epoch 0/0 (Cur. train loss: 0.6007):   3%|▎         | 112/4432 [09:52<6:12:45,  5.18s/it]









Train epoch 0/0 (Cur. train loss: 0.6007):   3%|▎         | 113/4432 [09:57<6:12:49,  5.18s/it]









Train epoch 0/0 (Cur. train loss: 0.5936):   3%|▎         | 113/4432 [09:57<6:12:49,  5.18s/it]









Train epoch 0/0 (Cur. train loss: 0.5936):   3%|▎         | 114/4432 [10:03<6:17:59,  5.25s/it]









Train epoch 0/0 (Cur. train loss: 0.6915):   3%|▎         | 114/4432 [10:03<6:17:59,  5.25s/it]









Train epoch 0/0 (Cur. train loss: 0.6915):   3%|▎         | 115/4432 [10:08<6:19:22,  5.27s/it]









Train epoch 0/0 (Cur. train loss: 0.7102):   3%|▎      

Train epoch 0/0 (Cur. train loss: 0.6435):   4%|▍         | 171/4432 [15:01<6:15:08,  5.28s/it]









Train epoch 0/0 (Cur. train loss: 0.6957):   4%|▍         | 171/4432 [15:01<6:15:08,  5.28s/it]









Train epoch 0/0 (Cur. train loss: 0.6957):   4%|▍         | 172/4432 [15:06<6:11:13,  5.23s/it]









Train epoch 0/0 (Cur. train loss: 0.7075):   4%|▍         | 172/4432 [15:06<6:11:13,  5.23s/it]









Train epoch 0/0 (Cur. train loss: 0.7075):   4%|▍         | 173/4432 [15:11<6:20:45,  5.36s/it]









Train epoch 0/0 (Cur. train loss: 0.6576):   4%|▍         | 173/4432 [15:11<6:20:45,  5.36s/it]









Train epoch 0/0 (Cur. train loss: 0.6576):   4%|▍         | 174/4432 [15:17<6:17:12,  5.32s/it]









Train epoch 0/0 (Cur. train loss: 0.6184):   4%|▍         | 174/4432 [15:17<6:17:12,  5.32s/it]









Train epoch 0/0 (Cur. train loss: 0.6184):   4%|▍         | 175/4432 [15:22<6:12:19,  5.25s/it]









Train epoch 0/0 (Cur. train loss: 0.7417):   4%|▍      

...

Train epoch 0/0 (Cur. train loss: 0.6648):  96%|█████████▌| 4238/4432 [6:44:05<14:04,  4.35s/it]









Train epoch 0/0 (Cur. train loss: 0.6648):  96%|█████████▌| 4239/4432 [6:44:09<13:53,  4.32s/it]









Train epoch 0/0 (Cur. train loss: 0.5653):  96%|█████████▌| 4239/4432 [6:44:10<13:53,  4.32s/it]









Train epoch 0/0 (Cur. train loss: 0.5653):  96%|█████████▌| 4240/4432 [6:44:14<13:54,  4.35s/it]









Train epoch 0/0 (Cur. train loss: 0.4968):  96%|█████████▌| 4240/4432 [6:44:14<13:54,  4.35s/it]









Train epoch 0/0 (Cur. train loss: 0.4968):  96%|█████████▌| 4241/4432 [6:44:18<13:55,  4.37s/it]









Train epoch 0/0 (Cur. train loss: 0.5490):  96%|█████████▌| 4241/4432 [6:44:18<13:55,  4.37s/it]









Train epoch 0/0 (Cur. train loss: 0.5490):  96%|█████████▌| 4242/4432 [6:44:22<13:38,  4.31s/it]









Train epoch 0/0 (Cur. train loss: 0.6855):  96%|█████████▌| 4242/4432 [6:44:23<13:38,  4.31s/it]









Train epoch 0/0 (Cur. train loss: 0.6855):  96

Train epoch 0/0 (Cur. train loss: 0.5732):  97%|█████████▋| 4298/4432 [6:48:45<09:34,  4.29s/it]









Train epoch 0/0 (Cur. train loss: 0.5732):  97%|█████████▋| 4299/4432 [6:48:50<09:32,  4.30s/it]









Train epoch 0/0 (Cur. train loss: 0.6793):  97%|█████████▋| 4299/4432 [6:48:50<09:32,  4.30s/it]









Train epoch 0/0 (Cur. train loss: 0.6793):  97%|█████████▋| 4300/4432 [6:48:56<10:40,  4.85s/it]









Train epoch 0/0 (Cur. train loss: 0.4873):  97%|█████████▋| 4300/4432 [6:48:56<10:40,  4.85s/it]









Train epoch 0/0 (Cur. train loss: 0.4873):  97%|█████████▋| 4301/4432 [6:49:00<10:22,  4.75s/it]









Train epoch 0/0 (Cur. train loss: 0.5676):  97%|█████████▋| 4301/4432 [6:49:00<10:22,  4.75s/it]









Train epoch 0/0 (Cur. train loss: 0.5676):  97%|█████████▋| 4302/4432 [6:49:05<10:08,  4.68s/it]









Train epoch 0/0 (Cur. train loss: 0.4644):  97%|█████████▋| 4302/4432 [6:49:05<10:08,  4.68s/it]









Train epoch 0/0 (Cur. train loss: 0.4644):  97

Train epoch 0/0 (Cur. train loss: 0.3895):  98%|█████████▊| 4358/4432 [6:53:57<06:52,  5.57s/it]









Train epoch 0/0 (Cur. train loss: 0.3895):  98%|█████████▊| 4359/4432 [6:54:01<06:19,  5.19s/it]









Train epoch 0/0 (Cur. train loss: 0.5407):  98%|█████████▊| 4359/4432 [6:54:01<06:19,  5.19s/it]









Train epoch 0/0 (Cur. train loss: 0.5407):  98%|█████████▊| 4360/4432 [6:54:06<05:55,  4.93s/it]









Train epoch 0/0 (Cur. train loss: 0.5024):  98%|█████████▊| 4360/4432 [6:54:06<05:55,  4.93s/it]









Train epoch 0/0 (Cur. train loss: 0.5024):  98%|█████████▊| 4361/4432 [6:54:15<07:29,  6.33s/it]









Train epoch 0/0 (Cur. train loss: 0.3672):  98%|█████████▊| 4361/4432 [6:54:15<07:29,  6.33s/it]









Train epoch 0/0 (Cur. train loss: 0.3672):  98%|█████████▊| 4362/4432 [6:54:32<11:05,  9.50s/it]









Train epoch 0/0 (Cur. train loss: 0.5598):  98%|█████████▊| 4362/4432 [6:54:32<11:05,  9.50s/it]









Train epoch 0/0 (Cur. train loss: 0.5598):  98

Train epoch 0/0 (Cur. train loss: 0.6153): 100%|█████████▉| 4418/4432 [6:59:52<01:29,  6.37s/it]









Train epoch 0/0 (Cur. train loss: 0.6153): 100%|█████████▉| 4419/4432 [6:59:57<01:14,  5.73s/it]









Train epoch 0/0 (Cur. train loss: 0.5410): 100%|█████████▉| 4419/4432 [6:59:57<01:14,  5.73s/it]









Train epoch 0/0 (Cur. train loss: 0.5410): 100%|█████████▉| 4420/4432 [7:00:01<01:04,  5.34s/it]









Train epoch 0/0 (Cur. train loss: 0.5109): 100%|█████████▉| 4420/4432 [7:00:01<01:04,  5.34s/it]









Train epoch 0/0 (Cur. train loss: 0.5109): 100%|█████████▉| 4421/4432 [7:00:05<00:54,  4.99s/it]









Train epoch 0/0 (Cur. train loss: 0.6613): 100%|█████████▉| 4421/4432 [7:00:05<00:54,  4.99s/it]









Train epoch 0/0 (Cur. train loss: 0.6613): 100%|█████████▉| 4422/4432 [7:00:09<00:47,  4.78s/it]









Train epoch 0/0 (Cur. train loss: 0.6332): 100%|█████████▉| 4422/4432 [7:00:09<00:47,  4.78s/it]









Train epoch 0/0 (Cur. train loss: 0.6332): 100











Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]









Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.27 Batches/s]


In [252]:
def predict_doc(basic_texts):
    save_dir = Path("testsave/doc_class")
    inf = Inferencer.load(save_dir, batch_size=1, num_processes=0)
    result = inf.inference_from_dicts(dicts=basic_texts)
    return result

In [253]:
df = pd.read_csv('test_parliament.tsv', sep='\t')

In [254]:
df

,text
0,Yesterday you kindly addressed the House befor...
1,"You will have noticed yesterday that, whatever..."
2,There is nothing we in the House can do about ...
3,This is the first Scottish questions since the...
4,Charles Kennedy was a regular participant at S...
...,...
8081,We are spending £1 billion over the four years...
8082,"We just had this question a few minutes ago, a..."
8083,"We have, in fact, made the process more freque..."
8084,Gentleman is mistaken in his implication: actu...


In [255]:
basic_texts = []

res_df = df.sample(500)
for index, row in res_df.iterrows():
    basic_texts.append({'text': row['text']})


In [256]:
res = predict_doc(basic_texts)

06/22/2021 21:40:37 - INFO - farm.utils -   Using device: CPU 
06/22/2021 21:40:37 - INFO - farm.utils -   Number of GPUs: 0
06/22/2021 21:40:37 - INFO - farm.utils -   Distributed Training: False
06/22/2021 21:40:37 - INFO - farm.utils -   Automatic Mixed Precision: None
06/22/2021 21:40:37 - INFO - farm.modeling.language_model -   
06/22/2021 21:40:37 - INFO - farm.modeling.language_model -   LOADING MODEL
06/22/2021 21:40:37 - INFO - farm.modeling.language_model -   =============
06/22/2021 21:40:37 - INFO - farm.modeling.language_model -   Model found locally at testsave/doc_class
06/22/2021 21:40:40 - INFO - farm.modeling.language_model -   Loaded testsave/doc_class
06/22/2021 21:40:40 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
06/22/2021 21:40:40 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
06/22/2021 21:40:40 - INFO - farm.modeling.prediction_head -   Prediction head initialized with si

Inferencing Samples:   6%|▌         | 28/500 [00:04<01:09,  6.79 Batches/s]









Inferencing Samples:   6%|▌         | 29/500 [00:04<01:03,  7.42 Batches/s]









Inferencing Samples:   6%|▌         | 30/500 [00:04<01:23,  5.61 Batches/s]









Inferencing Samples:   6%|▌         | 31/500 [00:04<01:15,  6.24 Batches/s]









Inferencing Samples:   6%|▋         | 32/500 [00:04<01:07,  6.95 Batches/s]









Inferencing Samples:   7%|▋         | 33/500 [00:04<01:15,  6.22 Batches/s]









Inferencing Samples:   7%|▋         | 34/500 [00:05<01:07,  6.92 Batches/s]









Inferencing Samples:   7%|▋         | 35/500 [00:05<01:01,  7.50 Batches/s]









Inferencing Samples:   7%|▋         | 36/500 [00:05<00:58,  7.96 Batches/s]









Inferencing Samples:   7%|▋         | 37/500 [00:05<01:04,  7.18 Batches/s]









Inferencing Samples:   8%|▊         | 38/500 [00:05<01:09,  6.69 Batches/s]









Inferencing Samples:   8%|▊         | 39/500 [00:05<01:03,  7.30 

Inferencing Samples:  34%|███▍      | 171/500 [00:27<01:00,  5.43 Batches/s]









Inferencing Samples:  34%|███▍      | 172/500 [00:27<00:58,  5.58 Batches/s]









Inferencing Samples:  35%|███▍      | 173/500 [00:27<00:57,  5.71 Batches/s]









Inferencing Samples:  35%|███▍      | 174/500 [00:27<01:02,  5.18 Batches/s]









Inferencing Samples:  35%|███▌      | 175/500 [00:27<01:02,  5.16 Batches/s]









Inferencing Samples:  35%|███▌      | 176/500 [00:27<00:54,  5.97 Batches/s]









Inferencing Samples:  35%|███▌      | 177/500 [00:28<00:48,  6.66 Batches/s]









Inferencing Samples:  36%|███▌      | 178/500 [00:28<00:44,  7.20 Batches/s]









Inferencing Samples:  36%|███▌      | 179/500 [00:28<00:41,  7.67 Batches/s]









Inferencing Samples:  36%|███▌      | 180/500 [00:28<00:39,  8.00 Batches/s]









Inferencing Samples:  36%|███▌      | 181/500 [00:28<00:37,  8.42 Batches/s]









Inferencing Samples:  36%|███▋      | 182/500 [00:28<0

Inferencing Samples:  63%|██████▎   | 314/500 [00:49<00:31,  5.84 Batches/s]









Inferencing Samples:  63%|██████▎   | 315/500 [00:50<00:27,  6.64 Batches/s]









Inferencing Samples:  63%|██████▎   | 316/500 [00:50<00:28,  6.44 Batches/s]









Inferencing Samples:  63%|██████▎   | 317/500 [00:50<00:25,  7.09 Batches/s]









Inferencing Samples:  64%|██████▎   | 318/500 [00:50<00:30,  5.97 Batches/s]









Inferencing Samples:  64%|██████▍   | 319/500 [00:50<00:33,  5.36 Batches/s]









Inferencing Samples:  64%|██████▍   | 320/500 [00:51<00:36,  4.98 Batches/s]









Inferencing Samples:  64%|██████▍   | 321/500 [00:51<00:37,  4.81 Batches/s]









Inferencing Samples:  64%|██████▍   | 322/500 [00:51<00:34,  5.15 Batches/s]









Inferencing Samples:  65%|██████▍   | 323/500 [00:51<00:32,  5.37 Batches/s]









Inferencing Samples:  65%|██████▍   | 324/500 [00:51<00:31,  5.51 Batches/s]









Inferencing Samples:  65%|██████▌   | 325/500 [00:51<0

Inferencing Samples:  91%|█████████ | 455/500 [01:13<00:06,  6.63 Batches/s]









Inferencing Samples:  91%|█████████ | 456/500 [01:13<00:06,  7.26 Batches/s]









Inferencing Samples:  91%|█████████▏| 457/500 [01:13<00:06,  6.82 Batches/s]









Inferencing Samples:  92%|█████████▏| 458/500 [01:13<00:06,  6.58 Batches/s]









Inferencing Samples:  92%|█████████▏| 459/500 [01:13<00:05,  7.22 Batches/s]









Inferencing Samples:  92%|█████████▏| 460/500 [01:13<00:05,  6.83 Batches/s]









Inferencing Samples:  92%|█████████▏| 461/500 [01:13<00:05,  7.46 Batches/s]









Inferencing Samples:  92%|█████████▏| 462/500 [01:13<00:04,  7.94 Batches/s]









Inferencing Samples:  93%|█████████▎| 463/500 [01:14<00:04,  8.28 Batches/s]









Inferencing Samples:  93%|█████████▎| 464/500 [01:14<00:04,  8.50 Batches/s]









Inferencing Samples:  93%|█████████▎| 465/500 [01:14<00:04,  7.50 Batches/s]









Inferencing Samples:  93%|█████████▎| 466/500 [01:14<0

In [257]:
res

[{'task': 'text_classification',
  'predictions': [{'start': None,
    'end': None,
    'context': 'It would be to allow an unlawful situation to have persisted from Tuesday night, whereby people with the right to register to vote were denied the ability to do so—and an arbitrary situation also, given that, because of the nature of queueing on a website, it would not be possible to be even-handed towards citizens.',
    'label': 'remain_labour',
    'probability': 0.5028303}]},
 {'task': 'text_classification',
  'predictions': [{'start': None,
    'end': None,
    'context': 'Friend made, that is why I particularly welcome the comment made by the right hon.',
    'label': 'leave_conservative',
    'probability': 0.8523133}]},
 {'task': 'text_classification',
  'predictions': [{'start': None,
    'end': None,
    'context': 'They then put all the other letters back in the mailbag and sent it on.',
    'label': 'remain_labour',
    'probability': 0.55071414}]},
 {'task': 'text_classifica

In [ ]:
res[0]["predictions"][0]["probability"]